In [1]:
!pip install transformers

In [2]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from datasets import load_dataset

In [3]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
# Load the English version of the STSB dataset
dataset = load_dataset("stsb_multi_mt", "en")

train_df = dataset['train'].to_pandas()
eval_df = dataset['dev'].to_pandas()
test_df = dataset['test'].to_pandas()

train_df['similarity_score'] /= 5.0
eval_df['similarity_score'] /= 5.0
test_df['similarity_score'] /= 5.0

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
def encode_sentence(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
        last_hidden_states = outputs.last_hidden_state
        sentence_embedding = last_hidden_states[:, 0, :]
    return sentence_embedding

In [6]:
def compute_similarity(sentence1, sentence2):
    
    embedding1 = encode_sentence(sentence1)
    embedding2 = encode_sentence(sentence2)

    similarity = cosine_similarity(embedding1, embedding2)
    
    return similarity.item()  



In [7]:
sim_score1 = compute_similarity("I had a bad day", "I had so much fun")
print(sim_score1)
sim_score2 = compute_similarity("I had a bad day", "Everything was terrible today")
print(sim_score2)

0.9059163331985474
0.9270343780517578


In [8]:
predicted_scores = test_df.apply(lambda row: compute_similarity(row['sentence1'], row['sentence2']), axis=1)

In [9]:
actual_scores = test_df['similarity_score']
mse = mean_squared_error(actual_scores, predicted_scores)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 0.23455665723155314
